In [ ]:
import tensorflow as tf
import os
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_data = tf.keras.utils.get_file(
    origin = dataset_url,
    fname = 'flower_photos.tgz',
    extract = True
)

base_directory = os.path.join(os.path.dirname(zip_data), 'flower_photos')

data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1.0 / 255.0,
    validation_split = 0.2,
    rotation_range = 20,
		zoom_range = 0.15,
		width_shift_range = 0.2,
		height_shift_range = 0.2,
		shear_range = 0.15,
		horizontal_flip = True,
		fill_mode = "nearest"
) 

batchsize = 64

training_data = data_generator.flow_from_directory(
    base_directory,
    target_size = (224, 224),
    batch_size = batchsize,
    subset = 'training'
)

testing_data = data_generator.flow_from_directory(
    base_directory,
    target_size = (224, 224),
    batch_size = batchsize,
    subset = 'validation'
)

228818944/228813984 [==============================] - 4s 0us/step
Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [ ]:
training_data.class_indices

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}

In [ ]:
import math
batch_size = 64
train_steps = training_data.samples // batch_size
train_steps 

45

In [ ]:
val_steps = testing_data.samples // batch_size
val_steps

11

In [ ]:
augmented_model = tf.keras.models.Sequential(
    [
     tf.keras.layers.Conv2D(
         128,
         (3, 3),
         activation = 'relu',
         input_shape = (224, 224, 3)
     ),
     tf.keras.layers.MaxPooling2D(2, 2),
     tf.keras.layers.Conv2D(
         128,
         (3, 3),
         activation = 'relu'
     ),
     tf.keras.layers.MaxPooling2D(2, 2),
     tf.keras.layers.Conv2D(
         64,
         (3, 3),
         activation = 'relu'
     ),
     tf.keras.layers.MaxPooling2D(2, 2),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(
         512,
         activation = 'relu'
     ),
     tf.keras.layers.Dense(
         256,
         activation = 'relu'
     ),
     tf.keras.layers.Dense(
         128,
         activation = 'relu'
     ),
     tf.keras.layers.Dense(
         5,
         activation = 'softmax'
     )
    ]
)

augmented_model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'sgd',
    metrics = ['accuracy']
)

augmented_model.fit(
    training_data,
    validation_data = testing_data,
    epochs = 25,
    steps_per_epoch = train_steps,
    validation_steps = val_steps,
)

Epoch 1/25
45/45 [==============================] - 63s 1s/step - loss: 1.5908 - accuracy: 0.2984 - val_loss: 1.5710 - val_accuracy: 0.3196
Epoch 2/25
45/45 [==============================] - 49s 1s/step - loss: 1.5296 - accuracy: 0.3523 - val_loss: 1.4851 - val_accuracy: 0.3366
Epoch 3/25
45/45 [==============================] - 49s 1s/step - loss: 1.4543 - accuracy: 0.3656 - val_loss: 1.4468 - val_accuracy: 0.3523
Epoch 4/25
45/45 [==============================] - 48s 1s/step - loss: 1.3699 - accuracy: 0.4073 - val_loss: 1.3301 - val_accuracy: 0.4219
Epoch 5/25
45/45 [==============================] - 48s 1s/step - loss: 1.3147 - accuracy: 0.4365 - val_loss: 1.2643 - val_accuracy: 0.4773
Epoch 6/25
45/45 [==============================] - 48s 1s/step - loss: 1.2313 - accuracy: 0.4783 - val_loss: 1.2187 - val_accuracy: 0.4943
Epoch 7/25
45/45 [==============================] - 48s 1s/step - loss: 1.1883 - accuracy: 0.4995 - val_loss: 1.2244 - val_accuracy: 0.4957
Epoch 8/25
45/45 [==

# Transfer Learning

Transfer learning (TL) is a research problem in machine learning (ML) that focuses on storing knowledge gained while solving one problem and applying it to a different but related problem. \
Source : https://en.wikipedia.org/wiki/Transfer_learning

![](https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs11042-019-07821-9/MediaObjects/11042_2019_7821_Fig1_HTML.png) \
Source : https://link.springer.com/article/10.1007/s11042-019-07821-9

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model

In [ ]:
resnet = ResNet101V2(
    input_shape = (224, 224, 3),
    include_top = False,
    weights = 'imagenet'
)

In [ ]:
for layer in resnet.layers:
  layer.trainable = False

In [ ]:
flattening_layer = Flatten()(resnet.output)

In [ ]:
prediction_layer = Dense(5, activation = 'softmax')(flattening_layer)

In [ ]:
model = Model(inputs = resnet.input, outputs = prediction_layer)

In [ ]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'sgd',
    metrics = ['accuracy']
)

In [ ]:
model.fit(
    training_data,
    epochs = 10,
    validation_data = testing_data,
    steps_per_epoch = train_steps,
    validation_steps = val_steps
)

Epoch 1/10
45/45 [==============================] - 103s 1s/step - loss: 20.1238 - accuracy: 0.6678 - val_loss: 9.4215 - val_accuracy: 0.7656
Epoch 2/10
45/45 [==============================] - 63s 1s/step - loss: 7.5117 - accuracy: 0.8003 - val_loss: 7.8858 - val_accuracy: 0.7969
Epoch 3/10
45/45 [==============================] - 64s 1s/step - loss: 5.2526 - accuracy: 0.8240 - val_loss: 4.7319 - val_accuracy: 0.8381
Epoch 4/10
45/45 [==============================] - 64s 1s/step - loss: 3.7543 - accuracy: 0.8630 - val_loss: 5.3541 - val_accuracy: 0.8239
Epoch 5/10
45/45 [==============================] - 64s 1s/step - loss: 4.2959 - accuracy: 0.8511 - val_loss: 7.1391 - val_accuracy: 0.8054
Epoch 6/10
45/45 [==============================] - 64s 1s/step - loss: 3.0039 - accuracy: 0.8814 - val_loss: 5.5554 - val_accuracy: 0.8395
Epoch 7/10
45/45 [==============================] - 64s 1s/step - loss: 2.6429 - accuracy: 0.8842 - val_loss: 6.1903 - val_accuracy: 0.8068
Epoch 8/10
45/45 [

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________